In [77]:
# import libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
from skimage import io
import time
import tifffile as tif

## Specify magic numbers

- Stacksize -> Number of images in the stack
- Data -> Path to datafile
- Coorfile -> Excel file (.xlsx) with coordinates of the layers in format
- Coorcount -> number of coordinates for each layer (only x or xy?)
- Layer-count -> number of layers to be segmented

In [78]:
# CHANGE WITH EVERY NEW DATASET
STACKSIZE = 1000
DATA = "D:/Hanna Analysis/Orientation_13_7_fused/13_7_fused_binary_cropped.tif"
COORFILE = np.asarray(pd.read_excel(r"D:/Hanna Analysis/Orientation_13_7_fused/LayerCoor_13_7_fused.xlsx")) # , sheet-name = "V1"
# number of x,y coordinate tuples in each line
LINE_COORCOUNT = 22
# total number of layers specified in coordinate file
LAYER_COUNT = 1


## Fixed numbers -> same for every dataset

In [79]:
# Fixed variables
# set the filling of the polygon, can be anything but 0, does not matter, because that is the part we do not need
EMPTY = 2
BLACK = 0
# number of x,y coordinate tuples in each polygon
COORCOUNT = 2*LINE_COORCOUNT

## Function for cropping stack into stack of only one layer

In [80]:
def layercropping(coorfile, layernumber, savefile):
    
    # open datafile an convert to stack, clear memory of large image file
    stack = io.imread(DATA)
    stackarr = np.asarray(stack)
    del stack
    
    #  local variables
    itrpltd_dist = []
    layertop = []
    layerbot = []
    layer_stack = []
    
    coorfile = coorfile[:,1:]

    # make polygon for top of stack (layertop) and bottom of stack (layerbot) by appending the upper and lower layer bound lines
    # add top line
    for n in range(LINE_COORCOUNT):
        layertop.append(tuple((coorfile[layernumber-1][n],coorfile[layernumber-1][LINE_COORCOUNT+n])))
        layerbot.append(tuple((coorfile[layernumber+(LAYER_COUNT)][n],coorfile[layernumber+(LAYER_COUNT)][LINE_COORCOUNT+n])))

    # add bottom line
    for n in range(LINE_COORCOUNT, 0,-1):
        layertop.append(tuple((coorfile[layernumber][n-1],coorfile[layernumber][n+LINE_COORCOUNT-1])))
        layerbot.append(tuple((coorfile[layernumber+(LAYER_COUNT+1)][n-1],coorfile[layernumber+(LAYER_COUNT+1)][n+LINE_COORCOUNT-1])))
    

    # calculate the difference between top and bottom polygon y-coordinates, the difference
    # by stack size -1 = number of images that need to be interpolated
    for n in range(COORCOUNT):
        diff = layertop[n][1]-layerbot[n][1]
        dist = diff/(STACKSIZE-1)
        itrpltd_dist.append(dist)

    # fill a stack of size stack_size (size of data stack) with coordinates of top layer -> will be changed for each image in next step
    arr1 = np.array(layertop)
    stack_coordinates = [arr1]*STACKSIZE
    stack_coord = np.asarray(stack_coordinates)

    # get an array for the interpolated coordinates for all images -> image coordinate - interpolated distance
    for n in range(STACKSIZE):        
        for i in range(COORCOUNT):         
            stack_coord[n][i][1] = stack_coord[n][i][1] - itrpltd_dist[i]*n

    # crop stack image by image to polygon and then create new stack with cropped images       
    for i in range(STACKSIZE):
        
        # select current image
        currentimage = stackarr[i]
        polygon = []

        # make calculated coordinates to tuples (only way they can be fed into the polygon drawing function)
        for n in range (COORCOUNT):
            coordpair = tuple(stack_coord[i][n])
            polygon.append(coordpair)
        
        # create empty image with same image as the data image
        maskIm = Image.new('L', (currentimage.shape[1], currentimage.shape[0]), BLACK)
        
        # draw polygon mask into empty image (fill polygon with 2s)
        ImageDraw.Draw(maskIm).polygon(polygon, outline=EMPTY, fill=EMPTY)
        arraymask = np.array(maskIm)
        
        # set selected image to 0 (black) where there is no filled polygon -> leaves the polygon data
        currentimage[arraymask!=EMPTY]=BLACK
        
        # append cropped image to new cropped image stack
        layer_stack.append(currentimage)
    
    # convert layer polygons to array
    arr_layertop = np.array(layertop)
    arr_layerbot = np.array(layerbot)
    
    # get for both top and bottom polygon the min and max y value
    MIN_top = min(arr_layertop[:,1])
    MAX_top = max(arr_layertop[:,1])
    MIN_bottom = min(arr_layerbot[:,1])
    MAX_bottom = max(arr_layerbot[:,1])

    # get overall min and max y value -> for whole stack
    MIN = min(MIN_top,MIN_bottom)
    MAX = max(MAX_top,MAX_bottom)

    # crop away top and bottom of image that does not contain data (data: between min & max)
    layer_stack = np.asarray(layer_stack)
    layer_stack_crop = layer_stack[:,MIN:MAX]

    # save layer stack as tiff file
    tif.imwrite(savefile, layer_stack_crop, bigtiff=True)

### Run layer crop function for each layer and save tif of layer stack

In [81]:
# run function for layers, input name the layer stack tiff should be saved as
layer1 = layercropping(COORFILE, 1, "layer1.tif")
#layer2 = layercropping(COORFILE, 2, "layer2.tif")
#layer3 = layercropping(COORFILE, 3, "layer3.tif")